In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import pickle

In [37]:
df = pd.read_csv('Churn_Modelling.csv')
df.head(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [38]:
### Removing irrelavent features
df = df.drop(['RowNumber','CustomerId','Surname'],axis = 1)

In [39]:
df.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0


In [40]:
### Encode gender categorical feature

label_encoder_gender = LabelEncoder()
df['Gender'] = label_encoder_gender.fit_transform(df['Gender'])
df.head(3)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1


In [41]:
df['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [42]:
## OneHot Encoding on Geography Column

onehot_encoder_geo = OneHotEncoder()
geo_encoder = onehot_encoder_geo.fit_transform(df[['Geography']])
geo_encoder.toarray()


array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [43]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [44]:
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(),columns=(onehot_encoder_geo.get_feature_names_out(['Geography'])))
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [45]:
## Concatinating dataframe with encoder_df

df = pd.concat([df.drop("Geography",axis = 1),geo_encoder_df],axis = 1)
df.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [46]:
## Save the encoders and scaler

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [47]:
## Divide the dataset into Independent and dependent dataset

X = df.drop('Exited',axis = 1)
y = df['Exited']

## Split the dataset into training and testing sets 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

### Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [48]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [49]:
## Save the scaler file

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

ANN IMPLEMENTATION


In [50]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
import datetime

In [51]:
## Build ANN model

model = Sequential([
    Dense(64,activation = 'relu',input_shape=(X_train.shape[1],)), ## HL1 layes connected with input layer
    Dense(32,activation = 'relu'), ## HL2
    Dense(1,activation = 'sigmoid') ## Output layer


])

In [52]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                832       
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [53]:
opt = tf.keras.optimizers.legacy.Adam(learning_rate = 0.01)
loss = tf.keras.losses.BinaryCrossentropy()

In [54]:
## Compile the model
model.compile(optimizer=opt,loss = "binary_crossentropy",metrics=['accuracy'])

In [55]:
## Set Up early stopping

early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [56]:
## Training the model

history = model.fit(
    X_train,y_train,validation_data = (X_test,y_test),epochs = 100,
    callbacks = early_stopping_callback
)

Epoch 1/100
250/250 [==============================] - 0s 786us/step - loss: 0.3989 - accuracy: 0.8386 - val_loss: 0.3617 - val_accuracy: 0.8520
Epoch 2/100
250/250 [==============================] - 0s 561us/step - loss: 0.3527 - accuracy: 0.8537 - val_loss: 0.3435 - val_accuracy: 0.8595
Epoch 3/100
250/250 [==============================] - 0s 544us/step - loss: 0.3463 - accuracy: 0.8571 - val_loss: 0.3524 - val_accuracy: 0.8535
Epoch 4/100
250/250 [==============================] - 0s 550us/step - loss: 0.3452 - accuracy: 0.8615 - val_loss: 0.3360 - val_accuracy: 0.8625
Epoch 5/100
250/250 [==============================] - 0s 549us/step - loss: 0.3429 - accuracy: 0.8587 - val_loss: 0.3402 - val_accuracy: 0.8570
Epoch 6/100
250/250 [==============================] - 0s 545us/step - loss: 0.3381 - accuracy: 0.8590 - val_loss: 0.3528 - val_accuracy: 0.8540
Epoch 7/100
250/250 [==============================] - 0s 549us/step - loss: 0.3354 - accuracy: 0.8621 - val_loss: 0.3523 - val_ac

In [57]:
model.save('model.h5')

/Users/anwar/Downloads/Customer_Churn_Prediction/venv/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
